Import Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats
from secrets import IEX_CLOUD_API_TOKEN

Import List of Stocks

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

Split List into groups for batch API calls

In [3]:
def split(list, n):
    for i in range(0, len(list), n):
        yield list[i : i+n]

groups = list(split(stocks['Ticker'], 100))
stock_symbols = []

for i in range(0, len(groups)):
    stock_symbols.append(','.join(groups[i]))

Create dataframe to store results

In [4]:
data_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'Momentum Score'
                ]

momentum_dataframe = pd.DataFrame(columns = data_columns)

Store API data in dataframe, using placeholder "none" for uncalcalculated data

In [5]:
for batch in stock_symbols:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    stock_data = requests.get(batch_api_call_url).json()
    
    for stock in batch.split(','):
        momentum_dataframe = momentum_dataframe.append(
                                        pd.Series([stock, 
                                                   stock_data[stock]['quote']['latestPrice'],
                                                   'none',
                                                   stock_data[stock]['stats']['year1ChangePercent'],
                                                   'none',
                                                   stock_data[stock]['stats']['month6ChangePercent'],
                                                   'none',
                                                   stock_data[stock]['stats']['month3ChangePercent'],
                                                   'none',
                                                   stock_data[stock]['stats']['month1ChangePercent'],
                                                   'none',
                                                   'none'
                                                   ], 
                                                  index = data_columns), 
                                        ignore_index = True)

momentum_dataframe
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,A,124.780,none,0.978856,none,0.268583,none,0.042818,none,-0.003733,none,none
1,AAL,22.300,none,1.225825,none,0.886136,none,0.40803,none,0.078979,none,none
2,AAP,186.820,none,1.364465,none,0.234918,none,0.134938,none,0.084855,none,none
3,AAPL,125.020,none,1.258695,none,0.154991,none,-0.063975,none,-0.027468,none,none
4,ABBV,105.630,none,0.74237,none,0.242348,none,0.027922,none,-0.014113,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.270,none,0.965136,none,0.221674,none,0.016321,none,0.033561,none,none
501,ZBH,162.910,none,0.985929,none,0.176247,none,0.050746,none,-0.036796,none,none
502,ZBRA,483.155,none,1.788809,none,0.883477,none,0.226543,none,-0.073737,none,none
503,ZION,53.680,none,1.316484,none,0.96956,none,0.251627,none,-0.011404,none,none
